In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import adhtools.utils
import pandas as pd
import os
import numpy as np

## Load corpus

In [2]:
import glob

book_files = glob.glob('/media/sf_VBox_Shared/Arabic/Fiqh/2018-11-13-Fiqh-stemmed-chapters/*.xml')
fnames = [os.path.basename(fn) for fn in book_files]
print(len(book_files))

10066


In [3]:
path_to_stopwordlist = '/media/sf_VBox_Shared/Arabic/arabic_stop-words_7-8-2018.txt'
external_stopwords = [line.strip() for line in open(path_to_stopwordlist, 'r', encoding='utf-8')]

In [4]:
corpus = adhtools.utils.corpus_str(book_files, analyzer=False, field='proposed_root')

## Make document-term matrix

In [5]:
vectorizer = CountVectorizer( stop_words=external_stopwords, min_df=2, max_df=0.9)
X = vectorizer.fit_transform(corpus)

In [6]:
feature_names = vectorizer.get_feature_names()
len(feature_names)

29755

In [7]:
# some example words from the first document
np.array(feature_names)[X[0].indices][:50]

array(['بقل', 'ربط', 'جدي', 'رفق', 'شرط', 'خوص', 'منع', 'قطع', 'بلد',
       'غرر', 'وإنا', 'طوق', 'زخرف', 'قرن', 'ذلل', 'سخر', 'سيأ', 'فوت',
       'حول', 'وسي', 'مدي', 'همز', 'كوف', 'فتح', 'وكآبة', 'همل', 'بسأ',
       'وعث', 'عرف', 'حرف', 'سرد', 'فذلك', 'سبع', 'وسع', 'قعد', 'قيأ',
       'شخص', 'هيأ', 'مضطجع', 'قذر', 'لحم', 'بات', 'أجر', 'عطي', 'فنادق',
       'فهم', 'غرب', 'رخص', 'رغث', 'قمل'], dtype='<U19')

In [8]:
most_imp_words = pd.Series(np.array(feature_names)[X.argmax(axis=1)].flatten())
most_imp_words.value_counts().head(20)

صلي    563
سلم    477
بيع    412
شهد    242
طلق    202
حرم    192
قتل    164
ملك    148
كتب    145
عتق    143
زكي    130
صوم    121
زوج    115
وصي    112
قرر    111
رهن    107
أبن    107
ثلث    105
دعا     99
ولد     98
dtype: int64

## Train model

In [9]:
nr_topics = 60
lda = LatentDirichletAllocation(n_components=nr_topics, random_state=0, max_iter=50)
document_topics = lda.fit_transform(X)

## Save files

In [10]:
fp_out = '/media/sf_VBox_Shared/Arabic/Analyses/topicmodelling/'

In [11]:
df_document_topics = pd.DataFrame(document_topics, index=fnames)
df_document_topics.to_csv(os.path.join(fp_out, 'document_topics_{}.csv'.format(nr_topics)))

In [12]:
lda.components_.shape

(60, 29755)

In [13]:
topic_words = pd.DataFrame(np.argsort(lda.components_, axis=1)[:,-10:][::-1])

In [14]:
topic_words = topic_words.applymap(lambda l: feature_names[l])

In [15]:
topic_words.to_csv(os.path.join(fp_out, 'topics_{}.csv'.format(nr_topics)))

In [16]:
topic_words

,0,1,2,3,4,5,6,7,8,9
0,ملك,ثبت,شهد,نكل,قضي,قوم,يود,يمن,حلف,دعا
1,صلي,دعا,فتح,عمل,فضل,سمي,سلم,غور,علم,علا
2,لخم,نظر,ثلث,سلم,قدم,غور,ملك,قسم,خلف,أبن
3,روي,نزل,أبن,أمر,نوس,نبأ,صلي,سلم,عمر,رسل
4,عرف,سمي,طلق,أكل,كلم,خرج,دخل,يمن,حلف,حنث
5,جوز,أمم,عود,ظهر,أذن,سفر,ركع,قوم,وقت,صلي
6,جوز,جبي,خرج,نفق,دفع,مول,فقر,زكي,عطي,صدق
7,قطع,قدر,نصف,جني,عشر,ذهب,حكم,ثلث,عقل,ودي
8,حكم,وجب,طعم,عتق,يمن,حرم,جزأ,كره,ظهر,كفر
9,قوم,ذكر,يوسف,عبر,سلم,خلف,حنف,علا,رحم,حقق
